# Playbook Explorer

This notebook is intended to be a live example of how to work with SysML v2 models at analysis-time. For these purposes, the following terms are introduced:
* An *interpretation* is the mapping of user model symbols (the "M1 model" in OMG-speak) into semantically-correct symbols that represent real world objects meant to conform to the model (the "M0" in OMG-speak). Interpretation semantics are inspired by https://www.w3.org/TR/owl2-direct-semantics/ and are mostly similar.
* A *sequence* for an interpretation contains *atoms* or *instances* that match to real world things. Reading a sequence from left to right provides a set of nested contexts for the atoms that is important to the interpretation. For example [Rocket#0, LS#3] is a 2-sequence to describe facts around the LS#3 atom when it is considered in context for Rocket#0. This is an important idea for the SysML time and occurrence model where one may want to see how values change under different conditions.

This is a notebook that walks through the random interpretation generator to help developers working on their own interpreters.

## Example Model

The model that is used for this example is a very simple parts model.

    package 'Simple Action Example' {
    import Base::Anything;
    import ScalarValues::*;
    package 'Fake Library' {
        action def Action {
            action subactions : Action [0..20];
        }
        connection def Connection {
            end source: Anything;
            end target: Anything;
        }
        connection def HappensDuring :> Connection {
            end earlierOccurrence: Action :>> Connection::source;
            end laterOccurrence: Action :>> Connection::target;
        }
    }
    
   
    
    action 'Build Burger': 'Fake Library'::Action {
        attribute 'Burger Kind' : String;
        first start;
        then action 'Place Buns' : 'Fake Library'::Action;
        then action 'Add Patty' : 'Fake Library'::Action;
        
        decide 'Next Topping';
        if 'Burger Kind' == "Cheeseburger" then 'Add Cheese';
        if 'Burger Kind' == "Hamburger" then 'Dress Burger';
        
        action 'Add Cheese' : 'Fake Library'::Action [0..1];
        action 'Dress Burger' : 'Fake Library'::Action [0..1];
        
        merge 'Finish Build';
        
        action 'Plate Burger' : 'Fake Library'::Action;
        then done;
        
        succession pattyToTopping : 'Fake Library'::HappensDuring first 'Add Patty' then 'Next Topping';
        succession cheeseToFinish : 'Fake Library'::HappensDuring first 'Add Cheese' then 'Finish Build';
        succession dressToFinish : 'Fake Library'::HappensDuring first 'Dress Burger' then 'Finish Build';
        
        succession 'Finish Build' then 'Plate Burger';
    }
}

## Imports

Import key modules, functions, and classes from the PyMBE library:

In [1]:
import pymbe.api as pm
from pymbe.label import get_label, get_label_for_id
import networkx as nx
import matplotlib.pyplot as plt

from pymbe.interpretation.interp_playbooks import *
from pymbe.interpretation.results import *

from pymbe.client import SysML2Client
from pymbe.query.metamodel_navigator import map_inputs_to_results

from pymbe.interpretation.calc_dependencies import generate_execution_order

import os

from pymbe.graph.lpg import SysML2LabeledPropertyGraph

## Key IDs

The unique identifiers below are useful references for walking through the interpretations generated in this notebook.

In [2]:
action_id = '2d1857b0-f54f-49c4-8231-592e92d0f8be'

add_cheese_action_id = '8ea260c0-9c6c-4aec-95d0-903cde4f83d6'

## Client Setup

The example here uses a local copy of the JSON file obtained by a GET operation on the SysML v2 API at:
http://sysml2-sst.intercax.com:9000/projects/a4f6a618-e4eb-4ac8-84b8-d6bcd3badcec/commits/c48aea9b-42fb-49b3-9a3e-9c39385408d7/elements?page[size]=5000

Create the client and load local data.

In [3]:
helper_client = SysML2Client()

path = "C:\\Users\\bjorn\\Documents\\Git\\pyMBE"
file_name = "\\tests\\data\\Simple Actions Model\\elements.json"

helper_client._load_disk_elements(path + str(file_name))

Create a graph representation of the model and load it into memory.

In [4]:
lpg = SysML2LabeledPropertyGraph()
lpg.update(helper_client.elements_by_id, False)

This is just a helper to make abbreviations more legible.

In [5]:
shorten_pre_bake = {
}

Create an interpretation of the Kerbal model using the random generator playbook. In general, this randomly selects:
- The ratios of partitioning abstract classifier sequence sets into concrete sets. For example, one draw may choose 2 liquid stages and 3 solids.
- The number of sequences to create for a given feature multiplicity. For example, draw 2 for a 0..8 engines : Liquid Engine PartUsage.

The playbook also attempts to make sequences created obey the Subsetting relationship (elements marked with subsets in M1 model should have their interpretation sequences entirely included within the interpretation sequences of the superset).

In [6]:
from pymbe.query.query import get_types_for_feature, get_features_typed_by_type, roll_up_multiplicity_for_type

In [7]:
get_types_for_feature(lpg, add_cheese_action_id)

['2d1857b0-f54f-49c4-8231-592e92d0f8be']

In [8]:
get_features_typed_by_type(lpg, action_id)

['a96a9d95-89cc-42d9-a5a8-5ba858448df5',
 'aebd4f41-d07d-46d6-9128-81a87813d7f3',
 '98f39ee5-32db-436a-9aff-01aaef0e3cb9',
 'f31aa2c6-db4b-47d2-9dfa-e520f2b9d50a',
 '8ea260c0-9c6c-4aec-95d0-903cde4f83d6',
 'f3ac7d60-609a-4904-80e6-3607ad668140']

In [9]:
banded_featuring_graph = lpg.get_projection("Expanded Banded Graph")

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:395: UserWarning: These edge types are not in the graph: {'FeatureValue'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


In [10]:
banded_roots = [
        node
        for node in banded_featuring_graph.nodes
        if banded_featuring_graph.out_degree(node) < 1
    ]
banded_roots

['87a7fec7-89c7-40f7-8428-bb0816f0ba04',
 'f3ac7d60-609a-4904-80e6-3607ad668140',
 'e2e4718c-f8e6-41c0-9976-97e499083491',
 'b165ab14-9d17-4df6-9be7-1b30b5a67b6b',
 'ad1b3c5a-2841-4c13-8492-4b051e7772b3',
 '09ccebfc-893c-478d-b0da-deeb6f1f8627',
 '7f4ea4e2-50ac-4afd-8eed-3cfde947e279',
 'a554eddf-f846-4536-8b05-fe87012e8e16']

In [11]:
from pymbe.interpretation.interp_playbooks import build_expression_sequence_templates
from pymbe.label import get_label_for_id

In [12]:
expr_seqs = build_expression_sequence_templates(lpg)
for seq in expr_seqs:
    sig_seq = []
    for item in seq:
        sig_seq.append(get_label_for_id(item, helper_client.elements_by_id))
    print(sig_seq)

['0 «Occurred LiteralInteger»', '$result']
['== ($x, $y) => $result', '$x: Anything']
['== ($x, $y) => $result', '$result']
['== ($x, $y) => $result', 'FRE.Burger Kind', '$result: String']
['== ($x, $y) => $result', '$y: Anything']
['0 «Occurred LiteralInteger»', '$result']
['== ($x, $y) => $result', '$x: Anything']
['== ($x, $y) => $result', '$result']
['== ($x, $y) => $result', 'FRE.Burger Kind', '$result: String']
['== ($x, $y) => $result', '$y: Anything']
['1 «Occurred LiteralInteger»', '$result']
['1 «Occurred LiteralInteger»', '$result']


In [13]:
from pymbe.query.metamodel_navigator import feature_multiplicity

In [14]:
feature_multiplicity(helper_client.elements_by_id[add_cheese_action_id], helper_client.elements_by_id, "lower")

0

In [15]:
from pymbe.interpretation.interp_playbooks import build_sequence_templates
feature_seqs = build_sequence_templates(lpg)

In [16]:
for seq in feature_seqs:
    for item in seq:
        print(get_label_for_id(item, helper_client.elements_by_id))
    print('======================')

Build Burger: Action
Add Cheese: Action
Build Burger: Action
Add Patty: Action
Build Burger: Action
Place Buns: Action
Build Burger: Action
Plate Burger: Action
Build Burger: Action
Burger Kind: String
Build Burger: Action
Dress Burger: Action
Build Burger: Action
start


In [17]:
roll_up_multiplicity_for_type(lpg, helper_client.elements_by_id[action_id], 'upper')

12

In [18]:
m0_interpretation = random_generator_playbook(
    helper_client,
    lpg,
    shorten_pre_bake
)

c:\users\bjorn\documents\git\pymbe\src\pymbe\graph\lpg.py:395: UserWarning: These edge types are not in the graph: {'ResultExpressionMembership'}.
  warn(f"These edge types are not in the graph: {mismatched_edge_types}.")


NotImplementedError: ('Cannot handle untyped features! Tried on start', ' id = 5d6e4fc3-0f61-4f10-86d2-d36463b93cd7')

To see how sequences are structured, the cell below renders sequences that show what type of atoms will fill particular positions in the sequence, as well as the maximum multiplicity (number of) sequences.

In [ ]:
from pymbe.query.query import roll_up_upper_multiplicity, roll_up_multiplicity_for_type

feat_sequences = build_sequence_templates(lpg=lpg)

total = 0
for seq in feat_sequences:
    print(str(pprint_single_id_list(seq, lpg.nodes)) + ", " + str(roll_up_upper_multiplicity(lpg, lpg.nodes[seq[-1]])))

## Calculation Results Shown

The following cells are a series of displays of relevant features in the interpretation.

Show all interpretation sequence sets (limited to length of 5).

In [ ]:
for print_line in pprint_interpretation(m0_interpretation, lpg.nodes):
    print(print_line)